In [ ]:
pip install tweepy
pip install nltk
pip install scikit-learn
pip install matplotlib
pip install seaborn

In [ ]:
import tweepy
import pandas as pd

api_key = '77bPxmrDKurTSB6ilGQ3eLfzQ'
api_key_secret = 'C04TfeS4BGJCRmUCK4rhmNgEG0rAKjzvH3j7Wpl3Bzwn4hIHDG'
access_token = '1851662064957415424-teS1YB5zBXNK2oLtqVlNOTRd64LOHC'
access_token_secret = 'EQG4bLV31WD8rB8w7dR7gdQtxeXfc8I51E5aQyRIRXb1v'

auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

def fetch_tweets(keyword, num_tweets):
    tweets = tweepy.Cursor(api.search_tweets, q=keyword, lang="en").items(num_tweets)
    tweet_list = [[tweet.text] for tweet in tweets]
    return pd.DataFrame(tweet_list, columns=["Tweet"])

data = fetch_tweets("women safety OR harassment OR #metoo", 1000000)

In [ ]:
data.head()

In [ ]:
column_names =  ['text']

In [ ]:
data.head()

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = re.sub(r'http\S+|@\S+|[^A-Za-z\s]', '', text)
    text = text.lower()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
    return text

data['Cleaned_Tweet'] = data['text'].apply(preprocess_text)

In [ ]:
data.head()

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

In [ ]:
def get_sentiment(text):
    score = sia.polarity_scores(text)

    if score['compound'] >= 0.05:
        return 'positive'
    elif score['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'

data['Sentiment'] = data['Cleaned_Tweet'].apply(get_sentiment)

In [ ]:
data.head()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle


vectorizer = TfidfVectorizer(max_features=2000)
X = vectorizer.fit_transform(data['Cleaned_Tweet'])
y = data['Sentiment']

with open('vectorizer.sav', 'wb') as f:
    pickle.dump(vectorizer, f)

print("Vectorizer saved successfully!")

In [ ]:
print(X)
print(y)

In [ ]:
from sklearn.svm import SVC
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = SVC(kernel='linear')
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.countplot(x='Sentiment', data=data)
plt.title("Sentiment Distribution in Women's Safety Tweets")
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score

X_train_prediction = model.predict(X_train)
print(X_train_prediction)
print(y_train)
training_data_accuracy = accuracy_score(y_train,X_train_prediction)

In [ ]:
print('accuracy score on training data =',training_data_accuracy)

In [ ]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(y_test,X_test_prediction)

In [ ]:
print('accuracy score on test data =',test_data_accuracy)

In [ ]:
import pickle

filename='trained_model.sav'
pickle.dump(model,open(filename,'wb'))

In [ ]:
loaded_model=pickle.load(open('/content/trained_model.sav','rb'))

In [ ]:
X_new = X_train[0]
print(X)

In [ ]:
prediction = loaded_model.predict(X_new)
print(prediction)

In [ ]:
nc = 'women are beautiful'

nc_vectorized = vectorizer.transform([nc])

predi = loaded_model.predict(nc_vectorized)
print(predi[0])

In [ ]:
pickle.dump(vectorizer, open('vectorizer1.sav', 'wb'))